## Ramdom forest

## Importar librerías

In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import optuna
from sklearn.model_selection import cross_val_score


c:\Users\Administrator\Documents\Proyecto seg clientes refuerzo\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargar el dataset

In [2]:
# Ruta del archivo
ruta = r'C:/Users/Administrator/Documents/Proyecto seg clientes refuerzo/Proyecto_seg_clientes/Data/teleCust1000t_listo.csv'

# Cargar el dataset
df = pd.read_csv(ruta)

# Ver las primeras filas y las columnas disponibles
print(df.head())
print(df.info())

   region  tenure  age  marital  address  income  ed  employ  retire  gender  \
0       2      13   44        1        9    64.0   4       5       0       0   
1       3      11   33        1        7   136.0   5       5       0       0   
2       3      68   52        1       24   116.0   1      29       0       1   
3       2      33   33        0       12    33.0   2       0       0       1   
4       2      23   30        1        9    30.0   1       2       0       0   

   reside  custcat  
0       2        1  
1       6        4  
2       2        3  
3       1        1  
4       4        3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   region   1000 non-null   int64  
 1   tenure   1000 non-null   int64  
 2   age      1000 non-null   int64  
 3   marital  1000 non-null   int64  
 4   address  1000 non-null   int64  
 5   income   1000 no

## Preparar las características (X) y la variable objetivo (y)
Seleccionamos las columnas correspondientes a las características y la variable objetivo.

In [3]:
# Separar características (X) y la variable objetivo (y)
X = df.drop(columns=['custcat'])  # Elimina la columna objetivo de las características
y = df['custcat']  # Variable objetivo

## Dividir el dataset en conjuntos de entrenamiento y prueba
Dividimos los datos en entrenamiento y prueba para evaluar el rendimiento del modelo.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Crear y entrenar el modelo de Random Forest
Creamos una instancia del modelo RandomForestClassifier y lo entrenamos con los datos de entrenamiento.

In [5]:
# Crear el modelo
model = RandomForestClassifier(random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

## Hacer predicciones
Usamos el modelo para predecir los valores del conjunto de prueba.

In [6]:
# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

## Evaluar el modelo
Calculamos la exactitud y generamos un informe de clasificación para evaluar el rendimiento.

In [7]:
# Exactitud del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud del modelo: {accuracy:.2f}')

# Informe de clasificación
print('Informe de clasificación:')
print(classification_report(y_test, y_pred))

Exactitud del modelo: 0.36
Informe de clasificación:
              precision    recall  f1-score   support

           1       0.42      0.45      0.43        60
           2       0.30      0.28      0.29        39
           3       0.42      0.44      0.43        55
           4       0.27      0.24      0.25        46

    accuracy                           0.36       200
   macro avg       0.35      0.35      0.35       200
weighted avg       0.36      0.36      0.36       200



## Ajustar hiperparámetros

In [8]:
model = RandomForestClassifier(
    n_estimators=100,      # Número de árboles
    max_depth=10,          # Profundidad máxima del árbol
    random_state=42
)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [9]:
# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [10]:
# Exactitud del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud del modelo: {accuracy:.2f}')

# Informe de clasificación
print('Informe de clasificación:')
print(classification_report(y_test, y_pred))

Exactitud del modelo: 0.38
Informe de clasificación:
              precision    recall  f1-score   support

           1       0.43      0.42      0.42        60
           2       0.29      0.26      0.27        39
           3       0.42      0.49      0.45        55
           4       0.32      0.30      0.31        46

    accuracy                           0.38       200
   macro avg       0.37      0.37      0.37       200
weighted avg       0.38      0.38      0.38       200



## Optimizar con Optuna

In [12]:
# Función objetivo para Optuna
def objective(trial):
    # Hiperparámetros a optimizar
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

# Modelo con los hiperparámetros sugeridos
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

# Validación cruzada para evaluar el modelo
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score

# Crear un estudio y optimizar
study = optuna.create_study(direction='maximize')  # Maximizar la exactitud
study.optimize(objective, n_trials=50)  # Realizar 50 pruebas

# Mejor hiperparámetro encontrado
print("Mejor conjunto de hiperparámetros:", study.best_params)
print("Mejor puntuación de validación cruzada:", study.best_value)


[I 2024-11-26 14:12:44,429] A new study created in memory with name: no-name-61825223-e6e6-48ed-8958-191bc8fb6bee
[I 2024-11-26 14:12:46,070] Trial 0 finished with value: 0.37499999999999994 and parameters: {'n_estimators': 86, 'max_depth': 50, 'min_samples_split': 4, 'min_samples_leaf': 7, 'max_features': None}. Best is trial 0 with value: 0.37499999999999994.
[I 2024-11-26 14:12:48,270] Trial 1 finished with value: 0.37374999999999997 and parameters: {'n_estimators': 125, 'max_depth': 24, 'min_samples_split': 3, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 0 with value: 0.37499999999999994.
[I 2024-11-26 14:12:51,522] Trial 2 finished with value: 0.38 and parameters: {'n_estimators': 257, 'max_depth': 33, 'min_samples_split': 16, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 2 with value: 0.38.
[I 2024-11-26 14:12:52,756] Trial 3 finished with value: 0.39125 and parameters: {'n_estimators': 115, 'max_depth': 44, 'min_samples_split': 2, 'min_samples_leaf':

Mejor conjunto de hiperparámetros: {'n_estimators': 198, 'max_depth': 14, 'min_samples_split': 12, 'min_samples_leaf': 9, 'max_features': 'log2'}
Mejor puntuación de validación cruzada: 0.4050000000000001


## Entrenar el modelo con los mejores hiperparámetros

In [13]:
# Entrenar el modelo con los mejores hiperparámetros
best_params = study.best_params
model = RandomForestClassifier(**best_params, random_state=42)
model.fit(X_train, y_train)

# Evaluar en el conjunto de prueba
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Exactitud en el conjunto de prueba: {accuracy:.2f}')


Exactitud en el conjunto de prueba: 0.41
